In [ ]:
!pip install arxiv transformers datasets torch sentencepiece rouge-score nltk spacy fastapi uvicorn

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 6.4 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a20d3da003795975b325108ed24daac27a2a645dc616dad4bee2968b84ead9b8
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=41cbb2ad42f7024c9e7dac1f6c7f9d29e894a94b9f54c9da3f37d8b925a77cf0
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built rouge-score sgmllib3k


In [2]:
# Hugging Face Transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments

# Dataset loading, it imports the "load_dataset" function from the Hugging Face library
# "datasets" provides a lot of preprocessed datasets to be used for ML and NLP tasks
from datasets import load_dataset

# Data Processing
import pandas as pd
import numpy as np

# NLP utilities
import nltk
import spacy

# For Evaluation
from rouge_score import rouge_scorer

# Downloads and loads the scientific papers dataset
# It stores the result of the function call into the var "dataset"
# 1st param is the "Library of research papers",
#   2nd param is the subset inside the "Library" only containing CS, Physics, Math papers that we will summarize later
dataset = load_dataset ("scientific_papers", "arxiv")

train_texts = dataset ['train'] ['article']
train_summs = dataset ['train'] ['abstract']



Exception ignored in: <function _xla_gc_callback at 0x7d8157b945e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 
